# CLIP Notebook
This is a jupyter notebook file to execute [CLIP](ontolearn.concept_learner.CLIP) and generate predictive results. We recommend you to see the [concept learners](../docs/usage/06_concept_learners.md) guide before continuing with the execution.
Also if you have not done it already, from the main directory "Ontolearn", run the commands for Datasets mentioned [here](https://ontolearn-docs-dice-group.netlify.app/usage/02_installation#download-external-files) to download the datasets.

In [1]:
import json
from ontolearn.knowledge_base import KnowledgeBase
from ontolearn.concept_learner import CLIP
from ontolearn.refinement_operators import ExpressRefinement
from ontolearn.learning_problem import PosNegLPStandard
from owlapy.owl_individual import OWLNamedIndividual, IRI
from ontolearn.utils import setup_logging


Open `uncle_lp.json` where we have stored the learning problem for the concept of 'Uncle' and the path to the 'family' ontology.

In [2]:
with open('uncle_lp.json') as json_file:
    settings = json.load(json_file)

Create an instance of the class `KnowledeBase` by using the path that is stored in `settings`.

In [3]:
kb = KnowledgeBase(path=settings['data_path'])

Retreive the IRIs of the positive and negative examples of Uncle from `settings` and create an instance of `PosNegLPStandard`. (more info about this [here](../docs/usage/06_concept_learners.md#configure-the-learning-problem))

In [4]:
examples = settings['Uncle']
p = set(examples['positive_examples'])
n = set(examples['negative_examples'])
typed_pos = set(map(OWLNamedIndividual, map(IRI.create, p)))
typed_neg = set(map(OWLNamedIndividual, map(IRI.create, n)))
lp = PosNegLPStandard(pos=typed_pos, neg=typed_neg)

Create a model of [CLIP](ontolearn.concept_learner.CLIP) and fit the learning problem to the model.

In [5]:
op = ExpressRefinement(knowledge_base=kb, use_inverse=False,
                          use_numeric_datatypes=False)

In [6]:
model = CLIP(knowledge_base=kb, path_of_embeddings="../CLIPData/family/embeddings/ConEx_entity_embeddings.csv",
             refinement_operator=op, load_pretrained=True, max_runtime=200)
model.fit(lp)


 Loaded length predictor!

 Loaded length predictor!

 Loaded length predictor!

 Loaded length predictor!

***** Predicted length: 5 *****

***** Predicted length: 5 *****


## Retrieve top 3 hypotheses and print them.

In [7]:
hypotheses = list(model.best_hypotheses(n=3))
[print(_) for _ in hypotheses]

<class 'ontolearn.search.OENode'> at 0x0304774	Male ⊓ (∀ hasParent.Grandparent)	Quality:0.90476	Heuristic:0.40407	Depth:2	H_exp:6	|RC|:7	|Indv.|:None
<class 'ontolearn.search.OENode'> at 0x0ca154a	Male ⊓ (∀ hasChild.Grandchild)	Quality:0.90476	Heuristic:0.36919	Depth:1	H_exp:7	|RC|:7	|Indv.|:None
<class 'ontolearn.search.OENode'> at 0x2adbb89	Male ⊓ (∀ hasChild.(¬Grandfather))	Quality:0.88889	Heuristic:0.39044	Depth:3	H_exp:6	|RC|:0	|Indv.|:None


[None, None, None]